# Import librairie #

In [1]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

## Charger les variables d'environnement ##

In [2]:
# Charger le .env depuis la racine du projet
from pathlib import Path
dotenv_path = r'C:\Penny\.env'
load_result = load_dotenv(dotenv_path=dotenv_path)
print(f"Chargement .env : {'[OK] Reussi' if load_result else '[ERREUR] Echec'}")

Chargement .env : ✓ Réussi


## Connexion Redshift Pennylane ##

In [3]:
conn = psycopg2.connect(
    host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
    port=5439,
    dbname='prod',
    user='u_289572',
    password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
)

print("Connexion Redshift etablie")

Connexion Redshift etablie


## Fonction helper : Connexion auto-reconnect ##

In [4]:
def get_active_connection():
    """Retourne une connexion active, la recrée si nécessaire"""
    global conn
    try:
        # Test si la connexion est active
        pd.read_sql("SELECT 1;", conn)
        return conn
    except:
        # Reconnexion si fermée
        print("⟳ Reconnexion Redshift...")
        conn = psycopg2.connect(
            host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
            port=5439,
            dbname='prod',
            user='u_289572',
            password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
        )
        print("✓ Reconnecté")
        return conn

# Utilisation : remplace "conn" par "get_active_connection()" dans tes requêtes
# Exemple :
df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())
print(f"Test réussi : {len(df)} ligne(s)")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_50264\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_50264\3220976796.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())


Test réussi : 1 ligne(s)


## Liste table Factures clients ##

In [5]:
# Tables customer invoices Pennylane
tables_extended = [
    'customer_invoices'
]

accessible_tables = []

for table in tables_extended:
    try:
        query = f"SELECT * FROM pennylane.{table} LIMIT 1;"
        df = pd.read_sql(query, conn)
        accessible_tables.append({
            'table_name': table,
            'columns': len(df.columns),
            'column_list': ', '.join(df.columns.tolist()[:5]) + '...'
        })
        print(f"✓ {table} ({len(df.columns)} colonnes)")
    except:
        pass

print(f"\n\n=== RÉSUMÉ: {len(accessible_tables)} tables accessibles ===")
df_accessible = pd.DataFrame(accessible_tables)
print(df_accessible.to_string(index=False))

C:\Users\ycloa\AppData\Local\Temp\ipykernel_50264\2201023932.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ customer_invoices (17 colonnes)


=== RÉSUMÉ: 1 tables accessibles ===
       table_name  columns                                           column_list
customer_invoices       17 id, created_date, issue_date, deadline, company_id...


## Liste colonnes factures clients Pennylane ##

In [6]:
# Afficher les colonnes customer invoices
tables_found = [
    'customer_invoices'
]

for table in tables_found:
    query = f"SELECT * FROM pennylane.{table} LIMIT 0;"  # LIMIT 0 = structure seulement
    df = pd.read_sql(query, conn)
    print(f"\n{'='*60}")
    print(f"Table: pennylane.{table}")
    print(f"{'='*60}")
    print(f"Colonnes ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")


Table: pennylane.customer_invoices
Colonnes (17):
   1. id
   2. created_date
   3. issue_date
   4. deadline
   5. company_id
   6. customer_id
   7. customer_name
   8. amount_eur
   9. amount_eur_excl_taxes
  10. currency
  11. label
  12. invoice_number
  13. source
  14. is_paid
  15. outstanding_balance
  16. created_at
  17. updated_at


C:\Users\ycloa\AppData\Local\Temp\ipykernel_50264\1432569361.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Chargement des données factures clients ##

In [7]:
# Charger factures clients
query_gl = "SELECT * FROM pennylane.customer_invoices;"
df_customer_invoices = pd.read_sql(query_gl, get_active_connection())

print(f"✓ Données chargées : {len(df_customer_invoices)} lignes, {len(df_customer_invoices.columns)} colonnes")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_50264\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_50264\1837426977.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer_invoices = pd.read_sql(query_gl, get_active_connection())


✓ Données chargées : 11 lignes, 17 colonnes


## Affichage des colonnes disponibles ##

In [8]:
print("Colonnes disponibles :")
for i, col in enumerate(df_customer_invoices.columns, 1):
    print(f"  {i:2d}. {col}")

Colonnes disponibles :
   1. id
   2. created_date
   3. issue_date
   4. deadline
   5. company_id
   6. customer_id
   7. customer_name
   8. amount_eur
   9. amount_eur_excl_taxes
  10. currency
  11. label
  12. invoice_number
  13. source
  14. is_paid
  15. outstanding_balance
  16. created_at
  17. updated_at


## Affichage type clonne ##

In [9]:
print(f"Types de données des {len(df_customer_invoices.columns)} colonnes :\n")
for i, (col, dtype) in enumerate(df_customer_invoices.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

Types de données des 17 colonnes :

   1. id                             → int64
   2. created_date                   → object
   3. issue_date                     → object
   4. deadline                       → object
   5. company_id                     → int64
   6. customer_id                    → int64
   7. customer_name                  → object
   8. amount_eur                     → float64
   9. amount_eur_excl_taxes          → float64
  10. currency                       → object
  11. label                          → object
  12. invoice_number                 → object
  13. source                         → object
  14. is_paid                        → bool
  15. outstanding_balance            → float64
  16. created_at                     → datetime64[ns]
  17. updated_at                     → datetime64[ns]


## Conversion des types de colonnes ##

In [10]:
# Conversion des types
df_customer_invoices = df_customer_invoices.astype({
    'created_date': 'datetime64[ns]',
    'issue_date': 'datetime64[ns]',
    'deadline': 'datetime64[ns]',
    'amount_eur': 'float64',
    'amount_eur_excl_taxes' : 'float64',
    'outstanding_balance': 'float64',
    'created_at': 'datetime64[ns]',
    'updated_at': 'datetime64[ns]'
})

# Convertir toutes les autres colonnes en string
columns_to_string = [col for col in df_customer_invoices.columns if col not in ['created_date','issue_date','deadline','amount_eur','amount_eur_excl_taxes','outstanding_balance','created_at','updated_at']]
df_customer_invoices[columns_to_string] = df_customer_invoices[columns_to_string].astype(str)

print("✓ Conversion effectuée\n")
print("Nouveaux types :")
for i, (col, dtype) in enumerate(df_customer_invoices.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

✓ Conversion effectuée

Nouveaux types :
   1. id                             → object
   2. created_date                   → datetime64[ns]
   3. issue_date                     → datetime64[ns]
   4. deadline                       → datetime64[ns]
   5. company_id                     → object
   6. customer_id                    → object
   7. customer_name                  → object
   8. amount_eur                     → float64
   9. amount_eur_excl_taxes          → float64
  10. currency                       → object
  11. label                          → object
  12. invoice_number                 → object
  13. source                         → object
  14. is_paid                        → object
  15. outstanding_balance            → float64
  16. created_at                     → datetime64[ns]
  17. updated_at                     → datetime64[ns]


## Connexion PostgreSQL local ##

In [11]:
# Connexion à PostgreSQL local (utilise les variables du .env)
conn_pg = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    port=int(os.getenv('POSTGRES_PORT')),
    dbname=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD')
)

print("✓ Connexion PostgreSQL établie")
print(f"  Database: {os.getenv('POSTGRES_DB')}")
print(f"  User: {os.getenv('POSTGRES_USER')}")
print(f"  Port: {os.getenv('POSTGRES_PORT')}")

✓ Connexion PostgreSQL établie
  Database: pennylane_data
  User: pennylane_user
  Port: 5433


## Import des données dans PostgreSQL avec SQLAlchemy ##

In [12]:
from sqlalchemy import create_engine

# Créer un engine SQLAlchemy
engine = create_engine(
    f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
)

print("✓ Engine SQLAlchemy créé")

✓ Engine SQLAlchemy créé


## Export du DataFrame vers PostgreSQL ##

## Export vers PostgreSQL local ##

In [ ]:
# Export DataFrame vers PostgreSQL local
df_customer_invoices.to_sql(
    name='customer_invoices',
    con=engine,
    schema='pennylane',
    if_exists='replace',
    index=False,
    method='multi',
    chunksize=1000
)

print(f"[OK] Table 'customer_invoices' exportee : {len(df_customer_invoices)} lignes")

# Fermer connexions
conn.close()
conn_pg.close()

print("[OK] Connexions fermees")